# [Do Deep Nets Really Need to be Deep?](https://arxiv.org/pdf/1312.6184)

## Introduction

A shallow neural network is capable of <ins>representing</ins> a more accurate function of the data through **Model Compression**, when compared to training the model on the original data set.

Therefore, the complexity of the underlying function and size of representation best used to <ins>learn</ins> the function are different.

**Gap**: Previous research assumed depth gives representational advantage, but wasn't questioned whether it was required in practice.

**Improvement**: Shows depth advantage is not about representational limits since a shallow net can approximate same function learned by deep net (**also sets foundation for original distillation paper**)

## Approach 

**Soft Targets**

It's better to train a student model on logits since different logits can map to same distribution when using softmax (technically losing information the complex model learned)

Also, softmax can lead to few large values relative to others, which would cause cross entropy to focus on them, ignoring others <br><br>

In [2]:
import torch
print(torch.nn.functional.softmax(torch.tensor([-10.0, 0.0, 10.0]), dim=-1))
print(torch.nn.functional.softmax(torch.tensor([10.0, 20.0, 30.0]), dim=-1))

tensor([2.0611e-09, 4.5398e-05, 9.9995e-01])
tensor([2.0611e-09, 4.5398e-05, 9.9995e-01])


Say softmax target is $[2.06e^{-9}, 4.53e^{-5}, 9.99e^{-1}]$ and prediction is $[\frac{1}{3}, \frac{1}{3}, \frac{1}{3}]$

$CE_{Loss} \approx -[3.0e^{-7} \ log(\frac{1}{3}) + 6.7e^{-3} \ log(\frac{1}{3}) + 9.9e^{-1} \ log(\frac{1}{3})]$

We can see most of the loss would come from largest dimension of the target, so the student model would focus mostly on matching that value.


Now consider a logits target of $[-10.0, 0.0, 10.0]$ and prediction is $[5, 5, 5]$

$MSE_{Loss} \approx \frac{1}{3}  [(-10.0 - 5)^2 + (0.0 - 5)^2 + (10.0 - 5)^2]  = \frac{1}{3}  [(-15)^2 + (5)^2 + (5)^2]$

Each dimension of target and prediction contribute substantially to the loss. 

## Result

**Why does Model Compression perform better than training on original dataset? (Even without using extra data)**

Can be due to: 
* Label noise: Teacher model can eliminate label errors by predicting those correctly (from generalization)
* Function complexity reduction: Teacher model simplifies region of true distribution that are hard to learn
* Uncertainty information: Teacher model soft targets provide more information than hard targets, e.g. confusable classes

**As accuracy of teacher model increases linearly, what happens to student model accuracy?**

The paper shows its nearly linear which means: 
* Improvements in the teacher lead to proportional improvements in the student
* The more accurate the teacher, the more efficient you can go in the smaller model
* The smaller models tried in the paper don't run out of capacity 

## Application

### Data

In [87]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [88]:
df = pd.read_csv('data/AirlineSatisfaction/prepared.csv')
target_column = "isSatisfied"
X = df.drop(columns=[target_column])
y = df[target_column]

In [89]:
X, X_unlabeled, y, y_unlabeled = train_test_split(X, y, test_size=0.8, random_state=42, stratify=y)

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [91]:
X_train.shape, X_test.shape

((16624, 23), (4156, 23))

In [152]:
# Convert data to tensor
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [153]:
# Convert data to tensor
X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.float32)

# Create DataLoader for batching
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

### Models

In [164]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class LargeNet(nn.Module):
    def __init__(self, input_size=(23), output_size=1, num_neurons=16):
        super(LargeNet, self).__init__()
        input_size = np.prod(input_size)
        self.fc1 = nn.Linear(input_size, num_neurons)
        self.fc2 = nn.Linear(num_neurons, num_neurons)
        self.fc3 = nn.Linear(num_neurons, num_neurons)
        self.out = nn.Linear(num_neurons, output_size)

    def forward(self, x):
        fc1 = F.relu(self.fc1(x))
        fc2 = F.relu(self.fc2(fc1))
        fc3 = F.relu(self.fc3(fc2))
        logits = self.out(fc3)
        return logits

class SmallNet(nn.Module):
    def __init__(self, input_size=(23), output_size=1, num_neurons=2):
        super(SmallNet, self).__init__()
        input_size = np.prod(input_size)
        self.fc1 = nn.Linear(input_size, num_neurons)
        self.fc2 = nn.Linear(num_neurons, output_size)

    def forward(self, x):
        fc1 = F.relu(self.fc1(x))
        logits = self.fc2(fc1)
        return logits

print(LargeNet())
print()
print(SmallNet())

LargeNet(
  (fc1): Linear(in_features=23, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (out): Linear(in_features=16, out_features=1, bias=True)
)

SmallNet(
  (fc1): Linear(in_features=23, out_features=2, bias=True)
  (fc2): Linear(in_features=2, out_features=1, bias=True)
)


In [165]:
from tqdm.notebook import tqdm

def train_model(model, optimizer, num_epochs=30):
    model = model.to(device)
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
    
        progress_bar = tqdm(total=len(train_loader))
    
        for inputs, labels in train_loader:
            inputs, labels = inputs.view(inputs.shape[0], -1).to(device), labels.to(device)
            optimizer.zero_grad()  # Zero the gradients
            
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs.view(-1), labels)  # Compute the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update the weights
            
            running_loss += loss.item() * inputs.size(0)
            
            progress_bar.update(1)
            progress_bar.set_description(f"Loss: {loss.item():.4f}")
        all_preds, all_labels = test_model(model)
        num_instances = len(all_preds)
        correct = (all_preds == all_labels).sum()
        accuracy =  correct / num_instances
        errors = num_instances - correct
        epoch_loss = running_loss / len(train_loader.dataset)
        print(
            f"Epoch {epoch+1}/{num_epochs}, "
            f"Loss: {epoch_loss:.4f}, "
            f"Accuracy: {accuracy:.4f}, Errors: {errors}"
        )

def test_model(model):
    model.eval()
    progress_bar = tqdm(total=len(test_loader))
    
    all_preds, all_labels = [], []
    for inputs, labels in test_loader:
        inputs, labels = inputs.view(inputs.shape[0], -1).to(device), labels.to(device)
        
        outputs = model(inputs)  # Forward pass
        outputs = nn.functional.sigmoid(outputs)
        preds = outputs > 0.5
        all_preds += list(preds)
        all_labels += list(labels)
        progress_bar.update(1)
    all_preds, all_labels = torch.tensor(all_preds), torch.tensor(all_labels)
    return all_preds, all_labels

In [166]:
device = "cuda" if torch.cuda.is_available else "cpu"
device

'cuda'

In [168]:
large_model = LargeNet()
optimizer = optim.Adam(large_model.parameters(), lr=4e-3)
criterion = nn.BCEWithLogitsLoss()
train_model(large_model, optimizer, num_epochs=30)

  0%|          | 0/520 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

Epoch 1/30, Loss: 0.2816, Accuracy: 0.9153, Errors: 352


  0%|          | 0/520 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

...

  0%|          | 0/130 [00:00<?, ?it/s]

Epoch 29/30, Loss: 0.0979, Accuracy: 0.9500, Errors: 208


  0%|          | 0/520 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

Epoch 30/30, Loss: 0.1021, Accuracy: 0.9504, Errors: 206


In [167]:
small_model = SmallNet()
optimizer = optim.Adam(small_model.parameters(), lr=4e-3)
criterion = nn.BCEWithLogitsLoss()
train_model(small_model, optimizer, num_epochs=30)

  0%|          | 0/520 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

Epoch 1/30, Loss: 0.4128, Accuracy: 0.8953, Errors: 435


  0%|          | 0/520 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

...

  0%|          | 0/130 [00:00<?, ?it/s]

Epoch 29/30, Loss: 0.1910, Accuracy: 0.9314, Errors: 285


  0%|          | 0/520 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

Epoch 30/30, Loss: 0.1902, Accuracy: 0.9288, Errors: 296


### Model Compression with Logits

In [169]:
def get_teacher_logits(model, dataloader):
    model.eval()
    all_logits = []

    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.view(inputs.shape[0], -1).to(device)
            logits = model(inputs)
            all_logits.append(logits.cpu())

    return torch.cat(all_logits, dim=0)

# Create Train DataLoader for batching
# Turn off shuffle to align with data ordering
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

teacher_logits = get_teacher_logits(large_model, train_loader)
teacher_logits.shape

torch.Size([16624, 1])

In [170]:
class DistillDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, teacher_logits):
        self.X = X
        self.y = y
        self.teacher_logits = teacher_logits

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.teacher_logits[idx]

In [188]:
def train_student_on_logits(student, optimizer, teacher_logits, num_epochs=30):
    student = student.to(device)
    logit_loss_fn = nn.MSELoss()

    # Create distillation dataset + loader
    distill_dataset = DistillDataset(X_train_tensor, y_train_tensor, teacher_logits)
    distill_loader = DataLoader(distill_dataset, batch_size=64, shuffle=True)

    for epoch in range(num_epochs):
        student.train()
        running_loss = 0.0

        progress_bar = tqdm(total=len(distill_loader))

        for inputs, labels, t_logits in distill_loader:
            inputs = inputs.view(inputs.shape[0], -1).to(device)
            labels = labels.to(device)
            t_logits = t_logits.to(device).view(-1)

            optimizer.zero_grad()

            # Student logits
            s_logits = student(inputs).view(-1)

            # RAW MSE loss on logits
            loss = logit_loss_fn(s_logits, t_logits)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

            progress_bar.update(1)
            progress_bar.set_description(f"Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(distill_loader.dataset)

        # ---------- SAME STATS AS train_model ----------
        all_preds, all_labels = test_model(student)
        num_instances = len(all_preds)
        correct = (all_preds == all_labels).sum()
        accuracy = correct / num_instances
        errors = num_instances - correct

        print(
            f"Epoch {epoch+1}/{num_epochs}, "
            f"Loss: {epoch_loss:.4f}, "
            f"Accuracy: {accuracy:.4f}, Errors: {errors}"
        )

In [190]:
student_model = SmallNet()
optimizer = optim.Adam(student_model.parameters(), lr=5e-3)
train_student_on_logits(student_model, optimizer, teacher_logits, num_epochs=35)

  0%|          | 0/260 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

Epoch 1/35, Loss: 50.5131, Accuracy: 0.8311, Errors: 702


  0%|          | 0/260 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

...

  0%|          | 0/130 [00:00<?, ?it/s]

Epoch 34/35, Loss: 11.4596, Accuracy: 0.9194, Errors: 335


  0%|          | 0/260 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

Epoch 35/35, Loss: 11.4767, Accuracy: 0.9155, Errors: 351
